In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from UliEngineering.EngineerIO import format_value
from si_prefix import si_format
import plecs_helper as helper
%matplotlib
%matplotlib inline

import seaborn as sns
sns.set_theme(context='notebook', style='darkgrid',
              palette='deep', font='sans-serif',
              font_scale=1, color_codes=True, rc=None)

# Imports and setup
from pint import UnitRegistry
from scipy.signal import find_peaks
from scipy.optimize import fsolve

# pandas display using scientific notation
# pd.set_option('display.float_format', lambda x: f'{x:.3e}')

# Make plots pretty
import seaborn as sns
sns.set_theme(context='notebook', style='darkgrid',
              palette='deep', font='sans-serif',
              font_scale=1, color_codes=True, rc=None)

# use pint
units = UnitRegistry()
units.default_format = "~P.2f"

Using matplotlib backend: <object object at 0x00000144A6511540>


C:\Users\ieykamp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Full Cycle Original Flyback Converter

In [ ]:
# Task 3 csv files: Vshunt and Vdrain
(df_task1_4V, tspan, tstep) = helper.read_rigol_csv("scope_data/Task1-1.csv", ch1 = "Vsh_4V", ch2 = "Vd_4V", shift = -0.75e-6)
(df_task1_7V, tspan, tstep) = helper.read_rigol_csv("scope_data/Task1-2.csv", ch1 = "Vsh_7V", ch2 = "Vd_7V")
(df_task1_9V, tspan, tstep) = helper.read_rigol_csv("scope_data/Task1-3.csv", ch1 = "Vsh_9V", ch2 = "Vd_9V")

# Task 4 csv files: Vinj and Vdd
(df_task2_4V, tspan, tstep) = helper.read_rigol_csv("scope_data/Task2-1.csv", ch1 = "Vinj_4V", ch2 = "Vdd_4V")
(df_task2_7V, tspan, tstep) = helper.read_rigol_csv("scope_data/Task2-2.csv", ch1 = "Vinj_7V", ch2 = "Vdd_7V", shift = 2e-6)
(df_task2_9V, tspan, tstep) = helper.read_rigol_csv("scope_data/Task2-3.csv", ch1 = "Vinj_9V", ch2 = "Vdd_9V", shift = 8e-6)

# Combine all variables into one for convenience
df = df_task1_4V.set_index("t").join([df_task1_7V.set_index("t"), df_task1_9V.set_index("t"), df_task2_4V.set_index("t"), df_task2_7V.set_index("t"), df_task2_9V.set_index("t")]).reset_index()

# Vshunt
df_envelope = df # df[(df["t"] > -2e-6) & (df["t"] < 2e-6)]
df_zoom = df[(df["t"] > -5e-6) & (df["t"] < 5e-6)]

fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, sharex = False, sharey = True, figsize = (12, 4))
fig.autofmt_xdate()
helper.axes_labels("Oscilloscope timestamp", "s", "Shunt Voltage (Vsh)", "V", title = "Shunt Voltage (Vsh)", ax = ax1)
ax1.plot(df_envelope["t"], df_envelope["Vsh_4V"], label = "Vsh_4V")
ax1.plot(df_envelope["t"], df_envelope["Vsh_7V"], label = "Vsh_7V")
ax1.plot(df_envelope["t"], df_envelope["Vsh_9V"], label = "Vsh_9V")
ax1.legend(loc = "lower left")
ax1.annotate('1', xy=(-6e-6, 150e-3), size=15, va="center", bbox=dict(boxstyle="round", fc="lightsteelblue", ec="none"))
ax1.annotate('2,3', xy=(5e-6, 100e-3), size=15, va="center", bbox=dict(boxstyle="round", fc="lightsteelblue", ec="none"))


helper.axes_labels("Oscilloscope timestamp", "s", "Shunt Voltage (Vsh)", "V", title = "Shunt Voltage (Vsh)", ax = ax2)
ax2.plot(df_zoom["t"], df_zoom["Vsh_4V"], label = "Vsh_4V")
ax2.plot(df_zoom["t"], df_zoom["Vsh_7V"], label = "Vsh_7V")
ax2.plot(df_zoom["t"], df_zoom["Vsh_9V"], label = "Vsh_9V")

Vshunt = np.array([150e-3, 260e-3, 330e-3])
# ax2.axhline(y = Vshunt[0], color = "black", linestyle = "dashed", label = f"Vsh = {int(Vshunt[0] * 1e3)}mV")
# ax2.axhline(y = Vshunt[1], color = "black", linestyle = "dashed", label = f"Vsh = {int(Vshunt[1] * 1e3)}mV")
# ax2.axhline(y = Vshunt[2], color = "black", linestyle = "dashed", label = f"Vsh = {int(Vshunt[2] * 1e3)}mV")
ax2.legend(loc = "lower left")

# plot duty cycle markers
duty_cycle_ts = [(-0.7e-6, 16e-6, 19.2e-6), (1.7e-6, 16e-6, 21.8e-6), (3.5e-6, 16e-6, 23.5e-6)]
for duty_cycle_marker_set in duty_cycle_ts:
    print(duty_cycle_marker_set)
    for duty_cycle_marker in duty_cycle_marker_set:
        ax1.axvline(x = duty_cycle_marker, linestyle = "dashed", color = "grey")
        pass

linear_ts = (-3e-6, 2e-6)
df_linreg = df_task1_9V[(df_task1_9V["t"] > linear_ts[0]) & (df_task1_9V["t"] < linear_ts[1])]

x = df_linreg["t"]
y = df_linreg["Vsh_9V"]
A = np.vstack([x, np.ones(len(x))]).T
a, b = np.linalg.lstsq(A, y, rcond=None)[0]
ax2.plot(df_linreg["t"], df_linreg["t"] * a + b, linestyle = "dashdot", color = "black", label = "Linear fit")
ax2.legend(loc = "lower left")